# Programming Assignment 05
Student: John Wu

In [2]:
import sys, os, nltk
import numpy as np, tokenHelper as tkn

In [5]:
fName = os.path.join(os.getcwd(), '19991220-Excite-QueryLog.utf8.tsv')

header = ['timestamp', 'userID', 'firstRank', 'query'] # variable names
raw = np.genfromtxt(fName, delimiter='\t', dtype=None, names=header, 
              comments=None, encoding='utf-8')

In [22]:
len(raw)/len(np.unique(raw['userID']))

4.607707030181962

## Simple Analysis

**Q1. What is the average number of queries per user id?**

In [23]:
print('On avg, %.2f queries per user.'%(len(raw)/len(np.unique(raw['userID']))))

On avg, 4.61 queries per user.


**Q2. Report the mean and median query length in both words and characters.**

In [ ]:
################################################################################

Simple Analysis (70 points)
Please answer any ten of the following questions (Q1 to Q13).
Q1. What is the average number of queries per user id?
Q2. Report the mean and median query length in both words and characters.
Q3. What percentage of queries are mixed case? All upper case? All lower case?
Q4. What percent of the time does a user request only the top 10 results?
Q5. What percent of unique queries are in the form of an explicit question (i.e., look for patterns such as starting with
Wh-words, or ending with a '?' symbol). What is the most common type of question?
Q6. What are the 20-most common queries issued?
Q7. What are the 20 most common non-stopwords appearing in queries?
Q8. What percent of queries contain stopwords like ‘and’, ‘the’, ‘of’, ‘in’, ‘at’?
Q9. What are the 10 most common non-stopwords appearing in queries that contain the word download?
Q10. What percentage of queries were asked by only one user?
Q11. Find 10 examples of misspelled words (but not 10 examples of the same misspelled word)
Q12. Which occurs in queries more often "Al Gore" or "Johns Hopkins"? "Johns Hopkins" or "John Hopkins"?
Q13. How often do URLs appear in queries?
Other Analysis (30 points)
Answer any three of the following questions (Q14 to Q20).
Q14. Estimate the percentage of queries that contain a person's name? (Or alternatively, a company name.)
Q15. Can you find addresses, phone numbers, and other identifiers in the log file? Is it likely that this web query log
puts anyone's privacy at risk? Justify your response.
Q16. How often is search engine “query” syntax used, like phrases in quotes, Boolean operators, or ‘+’ or ‘-‘ signs?
Q17. How often is a consecutive query a reformulation of the previous one? (Not the same query to greater depth.)
Q18. How does query volume change throughout the day?
Q19. What are the most popular websites mentioned in the queries?
Q20. Estimate the percentage of queries that are about sports?